In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.tree import  DecisionTreeClassifier 
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay